In [35]:
import pandas as pd
import numpy as np
import gcsfs
import sys
import xarray as xr
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

/tmp/ipykernel_478361/1529845526.py:9: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [36]:
df = pd.read_csv('https://cmip6.storage.googleapis.com/cmip6-zarr-consolidated-stores.csv')

In [37]:
# define a simple search on keywords
def search_df(df, verbose= False, **search):
    "search by keywords - if list, then match exactly, otherwise match as substring"
    keys = ['activity_id','institution_id','source_id','experiment_id','member_id', 'table_id', 'variable_id', 'grid_label']
    d = df
    for skey in search.keys():
        
        if isinstance(search[skey], str):  # match a string as a substring
            d = d[d[skey].str.contains(search[skey])]
        else:
            dk = []
            for key in search[skey]:       # match a list of strings exactly
                dk += [d[d[skey]==key]]
            d = pd.concat(dk)
            keys.remove(skey)
    if verbose:
        for key in keys:
            print(key,' = ',list(d[key].unique()))      
    return d

In [38]:
#dfp = search_df(df, experiment_id=['historical','ssp585'], table_id=['Omon','Amon'], variable_id=["uas","vas","tos"], grid_label=['gr'])
dfp = search_df(df,
                experiment_id=["historical","ssp245", "ssp585"],
                table_id=["Amon","Amon", "Amon", "Amon", "Omon", "SImon", "SImon", "SImon", "SImon","Amon","Omon","Amon","Amon","Amon"],
                variable_id=["prw","clt", "uas", "vas", "chl", "sithick", "siconc", "sisnthick", "sisnconc", "tas","tos","rsus","rsds","ps"],
                grid_label=['gn'])
#dfp = search_df(df, experiment_id=['piControl','historical','ssp126','ssp245','ssp370'],table_id=['Omon'], variable_id=["thetao"], grid_label=['gn'])


In [39]:
dm = dfp[["experiment_id","source_id","variable_id","member_id","grid_label"]].groupby(["member_id","source_id","experiment_id","grid_label"]).nunique()[["variable_id"]]

In [40]:
table = pd.DataFrame.pivot_table(dm, values="variable_id", index=["source_id","member_id"],
                                 columns=["grid_label","experiment_id"], aggfunc=np.sum, fill_value=0)
table[(table.values.sum(axis=1)==14*3)]

grid_label                      gn              
experiment_id           historical ssp245 ssp585
source_id     member_id                         
CanESM5       r10i1p2f1  14         14     14   
              r1i1p2f1   14         14     14   
              r2i1p2f1   14         14     14   
              r3i1p2f1   14         14     14   
              r4i1p2f1   14         14     14   
              r5i1p2f1   14         14     14   
              r6i1p2f1   14         14     14   
              r7i1p2f1   14         14     14   
              r8i1p2f1   14         14     14   
              r9i1p2f1   14         14     14   
MPI-ESM1-2-HR r1i1p1f1   14         14     14   
              r2i1p1f1   14         14     14   
MPI-ESM1-2-LR r10i1p1f1  14         14     14   
              r1i1p1f1   14         14     14   
              r2i1p1f1   14         14     14   
              r3i1p1f1   14         14     14   
              r4i1p1f1   14         14     14   
              r5i1p1f1   14         14     14   
              r6i1p1f1   14         14     14   
              r7i1p1f1   14         14     14   
              r8i1p1f1   14         14     14   
              r9i1p1f1   14         14     14   
UKESM1-0-LL   r1i1p1f2   14         14     14   
              r2i1p1f2   14         14     14   
              r3i1p1f2   14         14     14   
              r4i1p1f2   14         14     14   
              r8i1p1f2   14         14     14